In [ ]:
import numpy as np
import toml
import pandas as pd
import os
import scanpy as sc
from geneformer import TranscriptomeTokenizer
import loompy as lp

In [11]:
adata = sc.read_mtx('path/data/raw/dcm_hcm/matrix.mtx')

In [26]:
adata = adata.T

In [27]:
adata

AnnData object with n_obs × n_vars = 592689 × 36601

In [15]:
gene = pd.read_csv('path/data/raw/dcm_hcm/genes.tsv',sep='\t', header=None, index_col=1)
gene

,0
1,
MIR1302-2HG,ENSG00000243485
FAM138A,ENSG00000237613
OR4F5,ENSG00000186092
AL627309.1,ENSG00000238009
AL627309.3,ENSG00000239945
...,...
AC141272.1,ENSG00000277836
AC023491.2,ENSG00000278633
AC007325.1,ENSG00000276017


In [29]:
adata.var_names = list(gene.index)

In [21]:
cell = pd.read_csv('path/data/raw/dcm_hcm/barcodes.tsv',header=None)
cell

,0
0,TTCTTCCGTTCAACGT-1-0
1,CATCCACCATCTAACG-1-0
2,ACCCAAACAGCTAACT-1-0
3,AAGGAATCAACTGGTT-1-0
4,TACCCGTAGCGTGCTC-1-0
...,...
592684,TTATTGCGTCGGTGTC-1-79
592685,GTCACGGGTTGTATGC-1-79
592686,GTCATGATCTTTCGAT-1-79
592687,GCGATCGTCAGAGTGG-1-79


In [31]:
adata.obs_names = cell[0].values

Index(['TTCTTCCGTTCAACGT-1-0', 'CATCCACCATCTAACG-1-0', 'ACCCAAACAGCTAACT-1-0',
       'AAGGAATCAACTGGTT-1-0', 'TACCCGTAGCGTGCTC-1-0', 'GAAATGAGTATGGTTC-1-0',
       'CACTTCGCATCCCACT-1-0', 'AGACCCGGTATGTCCA-1-0', 'AACAACCCACCCTGTT-1-0',
       'GAGACCCTCCCGAGGT-1-0',
       ...
       'AAAGAACGTACGTGAG-1-79', 'TACTTACCAGGTCAGA-1-79',
       'AGGAATACATACCATG-1-79', 'CTATCTAGTCCAAGAG-1-79',
       'CATCCGTTCGCCGTGA-1-79', 'TTATTGCGTCGGTGTC-1-79',
       'GTCACGGGTTGTATGC-1-79', 'GTCATGATCTTTCGAT-1-79',
       'GCGATCGTCAGAGTGG-1-79', 'GCCCGAACAGAGAAAG-1-79'],
      dtype='object', length=592689)

In [20]:
meta = pd.read_csv('path/data/raw/dcm_hcm/DCM_HCM_MetaData_V1.txt', sep='\t', index_col='NAME')
meta

/root/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3012: DtypeWarning: Columns (8,9,10,11,12,13) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,biosample_id,donor_id,cell_type,cell_type__ontology_label,sex,cell_type_leiden06,sub_cluster,n_umi,n_genes,cellranger_percent_mito,...,entropy,doublet_score,species,species__ontology_label,disease,disease__ontology_label,organ,organ__ontology_label,library_preparation_protocol,library_preparation_protocol__ontology_label
NAME,,,,,,,,,,,,,,,,,,,,,
TYPE,group,group,group,group,group,group,group,numeric,numeric,numeric,...,numeric,numeric,group,group,group,group,group,group,group,group
TTCTTCCGTTCAACGT-1-0,LV_1622_2_nf,P1622,CL_0000746,cardiac muscle cell,male,00.Cardiomyocyte_I,CM-X1,15815.0,4632,0.0004308222560212016,...,7.503471227832365,0.0918918918918919,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0002084,heart left ventricle,EFO_0009922,10x 3' v3
CATCCACCATCTAACG-1-0,LV_1622_2_nf,P1622,CL_0000746,cardiac muscle cell,male,00.Cardiomyocyte_I,CM-HHATL,15546.0,4673,0.0005042546545155346,...,7.582058261621526,0.10172744721689062,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0002084,heart left ventricle,EFO_0009922,10x 3' v3
ACCCAAACAGCTAACT-1-0,LV_1622_2_nf,P1622,CL_0000746,cardiac muscle cell,male,00.Cardiomyocyte_I,CM-HHATL,14983.0,4432,0.0007832898409105837,...,7.466404841161381,0.0896184560780834,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0002084,heart left ventricle,EFO_0009922,10x 3' v3
AAGGAATCAACTGGTT-1-0,LV_1622_2_nf,P1622,CL_0000746,cardiac muscle cell,male,00.Cardiomyocyte_I,CM-X1,14995.0,4483,0.0007187193841673434,...,7.452265652814673,0.071875,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0002084,heart left ventricle,EFO_0009922,10x 3' v3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTATTGCGTCGGTGTC-1-79,LV_1561_2_nf,P1561,CL_0010008,cardiac endothelial cell,male,02.Endothelial_I,EC-Lymphatic,403.0,291,0.003145,...,6.691553,0.038961,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0002084,heart left ventricle,EFO_0009922,10x 3' v3
GTCACGGGTTGTATGC-1-79,LV_1561_2_nf,P1561,CL_0010008,cardiac endothelial cell,male,02.Endothelial_I,EC-PKD1L1,494.0,349,0.004847,...,6.791491,0.032258,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0002084,heart left ventricle,EFO_0009922,10x 3' v3
GTCATGATCTTTCGAT-1-79,LV_1561_2_nf,P1561,CL_0010008,cardiac endothelial cell,male,02.Endothelial_I,EC-Lymphatic,502.0,404,0.004918,...,7.409244,0.012023,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0002084,heart left ventricle,EFO_0009922,10x 3' v3


In [33]:
adata.obs = meta.iloc[1:]
adata.obs

,biosample_id,donor_id,cell_type,cell_type__ontology_label,sex,cell_type_leiden06,sub_cluster,n_umi,n_genes,cellranger_percent_mito,...,entropy,doublet_score,species,species__ontology_label,disease,disease__ontology_label,organ,organ__ontology_label,library_preparation_protocol,library_preparation_protocol__ontology_label
NAME,,,,,,,,,,,,,,,,,,,,,
TTCTTCCGTTCAACGT-1-0,LV_1622_2_nf,P1622,CL_0000746,cardiac muscle cell,male,00.Cardiomyocyte_I,CM-X1,15815.0,4632,0.0004308222560212016,...,7.503471227832365,0.0918918918918919,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0002084,heart left ventricle,EFO_0009922,10x 3' v3
CATCCACCATCTAACG-1-0,LV_1622_2_nf,P1622,CL_0000746,cardiac muscle cell,male,00.Cardiomyocyte_I,CM-HHATL,15546.0,4673,0.0005042546545155346,...,7.582058261621526,0.10172744721689062,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0002084,heart left ventricle,EFO_0009922,10x 3' v3
ACCCAAACAGCTAACT-1-0,LV_1622_2_nf,P1622,CL_0000746,cardiac muscle cell,male,00.Cardiomyocyte_I,CM-HHATL,14983.0,4432,0.0007832898409105837,...,7.466404841161381,0.0896184560780834,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0002084,heart left ventricle,EFO_0009922,10x 3' v3
AAGGAATCAACTGGTT-1-0,LV_1622_2_nf,P1622,CL_0000746,cardiac muscle cell,male,00.Cardiomyocyte_I,CM-X1,14995.0,4483,0.0007187193841673434,...,7.452265652814673,0.071875,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0002084,heart left ventricle,EFO_0009922,10x 3' v3
TACCCGTAGCGTGCTC-1-0,LV_1622_2_nf,P1622,CL_0000746,cardiac muscle cell,male,00.Cardiomyocyte_I,CM-HHATL,14797.0,4690,0.00046274872147478163,...,7.6090815967688155,0.06373626373626375,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0002084,heart left ventricle,EFO_0009922,10x 3' v3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTATTGCGTCGGTGTC-1-79,LV_1561_2_nf,P1561,CL_0010008,cardiac endothelial cell,male,02.Endothelial_I,EC-Lymphatic,403.0,291,0.003145,...,6.691553,0.038961,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0002084,heart left ventricle,EFO_0009922,10x 3' v3
GTCACGGGTTGTATGC-1-79,LV_1561_2_nf,P1561,CL_0010008,cardiac endothelial cell,male,02.Endothelial_I,EC-PKD1L1,494.0,349,0.004847,...,6.791491,0.032258,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0002084,heart left ventricle,EFO_0009922,10x 3' v3
GTCATGATCTTTCGAT-1-79,LV_1561_2_nf,P1561,CL_0010008,cardiac endothelial cell,male,02.Endothelial_I,EC-Lymphatic,502.0,404,0.004918,...,7.409244,0.012023,NCBITaxon_9606,Homo sapiens,PATO_0000461,normal,UBERON_0002084,heart left ventricle,EFO_0009922,10x 3' v3


In [ ]:
data_root = 'path'
select_cell = []
for i in range(42):
    select_cell.append(pd.read_csv(data_root+'data/processed/dcm_hcm/subset_%d/cell_names.csv'%i, index_col=0)['x'].values)
select_cell = np.concatenate(select_cell)
select_cell = ["".join(i.split('_')[1:]) for i in select_cell]
select_cell

In [44]:
adata[select_cell]

View of AnnData object with n_obs × n_vars = 523369 × 36601
    obs: 'biosample_id', 'donor_id', 'cell_type', 'cell_type__ontology_label', 'sex', 'cell_type_leiden06', 'sub_cluster', 'n_umi', 'n_genes', 'cellranger_percent_mito', 'exon_prop', 'entropy', 'doublet_score', 'species', 'species__ontology_label', 'disease', 'disease__ontology_label', 'organ', 'organ__ontology_label', 'library_preparation_protocol', 'library_preparation_protocol__ontology_label'

In [73]:
data_root = 'path'
row_attrs = {
    "Gene": np.array(adata[select_cell].var_names),
    'ensembl_id':np.array(adata[select_cell].var['ensembl_id']),
}

col_attrs = {
    "CellID": np.array(adata[select_cell].obs_names) ,
    "n_genes": np.array(adata[select_cell].obs['n_genes']).astype(np.float64),
    "n_counts": np.array(adata[select_cell].obs['n_umi']).astype(np.float64),
}
# os.mkdir('/opt/data/private/xx/code/MIRACLE/data/processed/geneformer/')
lp.create('/opt/data/private/xx/code/MIRACLE/data/processed/geneformer/dcm_hcm.loom', layers=adata[select_cell].X.transpose(), row_attrs=row_attrs, col_attrs=col_attrs)


In [ ]:
data_path = 'path/data/processed/geneformer/'
data_path_out ='path/data/processed/geneformer/tokenized_data'

tk = TranscriptomeTokenizer({}, nproc=4)
tk.tokenize_data(data_path, data_path_out, 'geneformer')

In [75]:
from geneformer.in_silico_perturber import load_from_disk, forward_pass_single_cell
from tqdm.notebook import trange
import os
import pandas as pd
os.environ['CUDA_VISIBLE_DEVICES']='1'
import numpy as np
import torch
import math
import matplotlib.pyplot as plt
import scanpy as sc

from transformers import BertForMaskedLM
def load_model(model_directory):
    model = BertForMaskedLM.from_pretrained(model_directory, 
                                            output_hidden_states=True,
                                            output_attentions=False)
    # put the model in eval mode for fwd pass
    model.eval()
    model = model.to("cuda:0")
    return model

In [76]:
model_directory = 'path/comparison/Geneformer'
model = load_model(model_directory)
def get_emb(data_path):
    data = load_from_disk(data_path)
    emb = []
    for i in trange(len(data)):
        example_cell = data.select([i])
        original_emb = forward_pass_single_cell(model, example_cell, -1).detach().cpu()
        cell_emb = torch.mean(original_emb,dim=0,keepdim=True)
        emb.append(cell_emb.tolist()[0])
    return emb

In [ ]:
result_dir = 'path/result/dcm_hcm/geneformer/default/predict/'
emb = get_emb('path/data/processed/geneformer/tokenized_data/geneformer.dataset')
os.makedirs(result_dir)
pd.DataFrame(emb).to_csv(result_dir+'00.csv', index=None, header=None)

In [78]:
pd.DataFrame(emb).to_csv(result_dir+'00.csv', index=None, header=None)

In [82]:
adata2 = sc.AnnData(np.array(emb))
adata2

/tmp/ipykernel_8497/2629862490.py:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata2 = sc.AnnData(np.array(emb))


AnnData object with n_obs × n_vars = 523369 × 256

In [83]:
sc.pp.pca(adata2)

In [84]:
sc.pp.neighbors(adata2)

In [86]:
sc.tl.umap(adata2)

In [ ]:
adata2.obs = adata[select].obs